In [ ]:
import os
os.chdir("/opt/project")

import psutil
from pathlib import Path
from functools import partial
from contextlib import suppress

import h5py
import numpy as np
import pandas as pd
import xarray as xr
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import (precision_score, recall_score, f1_score,
                             precision_recall_curve, average_precision_score)

import keras
keras.mixed_precision.set_global_policy("mixed_float16")

from model import get_model
from datasets import ECGSequence

In [ ]:
LOGS_PATH = Path("logs")
MODELS_PATH = Path("models")
FIGURES_PATH = Path("figures")
PREDICTIONS_PATH = Path("predictions")

TEST_DATA_PATH = Path("data/test")
INPUT_DATA_PATH = Path("data/train/inputs")
LABEL_DATA_PATH = Path("data/train/labels")
TEST_ANNOTATIONS_PATH = TEST_DATA_PATH / "annotations"

input_files = os.listdir(INPUT_DATA_PATH)
label_files = os.listdir(LABEL_DATA_PATH)

In [ ]:
def align_entries(indices: np.ndarray, data: np.ndarray, num_classes=6):
    ydict = {}
    for item in data:
        ydict[item[0]] = item

    result = np.empty(shape=(len(indices), num_classes), dtype=object)
    for index, value in enumerate(indices):
        with suppress(KeyError):
            result[index] = ydict[str(value)][4: 4 + num_classes] == "True"

    return result

In [ ]:
def process_and_export_file_data(input_file: str, output_file: str):
    y = pd.read_csv(LABEL_DATA_PATH.parent / "exams.csv", dtype=object).values
    with h5py.File(INPUT_DATA_PATH / f"{input_file}", "r+") as file:
        x, ids = file["tracings"], file["exam_id"]
        y_curr = align_entries(ids, y)

        # Log info
        print(f" FILE: {input_file} -> {output_file} ".center(60, "*"))
        print(f"X SHAPE: {x.shape}".center(60, " "))
        print(f"Y SHAPE: {y_curr.shape}".center(60, " "))
        print(f"I SHAPE: {ids.shape}\n".center(60, " "))

        # Save labels
        pd.DataFrame(y_curr).astype(int).to_csv(
            LABEL_DATA_PATH / output_file,
            sep=",", encoding="utf-8", index=False, header=True
        )

In [ ]:
for i in range(len(input_files)):
    if not f"exams_part{i}.hdf5" in input_files: break
    process_and_export_file_data(f"exams_part{i}.hdf5", f"exams_part{i}.csv")

In [ ]:
# Model settings
val_split = 0.02
dataset_name = "tracings"
model_id = len(os.listdir("models")) - 1
worker_num = psutil.cpu_count(logical=True) - 2

# Optimization settings
lr = 0.001
batch_size = 64
opt = keras.optimizers.Adam(lr)
loss = keras.losses.BinaryCrossentropy()

callbacks = [
    # Learning Optimizers
    keras.callbacks.EarlyStopping(patience=9, min_delta=0.00001),
    keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.1,
                                      patience=7, min_lr=lr / 100),
    # Logs
    keras.callbacks.TensorBoard(log_dir=LOGS_PATH, write_graph=False),
    keras.callbacks.CSVLogger(LOGS_PATH / "training.log", append=False),
    # Checkpoints
    keras.callbacks.ModelCheckpoint(MODELS_PATH / f"backup/model_last_{model_id}.keras"),
    keras.callbacks.ModelCheckpoint(MODELS_PATH / f"backup/model_best_{model_id}.keras"),
]

train_seq, valid_seq = ECGSequence.get_train_and_val(
    [INPUT_DATA_PATH / file for file in input_files],
    [LABEL_DATA_PATH / file for file in label_files],
    dataset_name, batch_size, val_split,
    workers=worker_num, use_multiprocessing=True
)

# If you are continuing an interrupted section, uncomment line bellow:
# model = keras.models.load_model(PATH_TO_MODEL, compile=False)
model = get_model(train_seq.n_classes)
model.compile(loss=loss, optimizer=opt)

In [ ]:
# Train neural network
# If you are continuing an interrupted section change initial epoch
history = model.fit(train_seq,
                    epochs=70,
                    initial_epoch=0,
                    callbacks=callbacks,
                    validation_data=valid_seq,
                    verbose=1)
# Save final result
model.save(MODELS_PATH / f"model_{model_id}.keras")

In [ ]:
# Predictions of the model on the test set
seq = ECGSequence([TEST_DATA_PATH / "ecg_tracings.hdf5"], None, dataset_name, batch_size=1)
y_score = model.predict(seq, verbose=1)

np.save(PREDICTIONS_PATH / f"predictions_{model_id}.npy", y_score)

In [ ]:
model_id = 1

In [ ]:
# Auxiliary functions

def get_scores(y_true, y_nn, score_func):
    scores_arr = []
    for name, fun in score_func.items():
        scores_arr += [[fun(y_true[:, k], y_nn[:, k])
                    for k in range(np.shape(y_true)[1])]]
    return np.array(scores_arr).T


def get_optimal_precision_recall(y_true, y_nn):
    """Find precision and recall values that maximize f1 score."""
    opt_recall = []
    opt_precision = []
    opt_threshold = []
    for k in range(np.shape(y_true)[1]):
        # Get precision-recall curve
        precision, recall, thresh = precision_recall_curve(y_true[:, k], y_nn[:, k])
        f1 = np.nan_to_num(2 * precision * recall / (precision + recall))

        # Select threshold that maximize f1 score
        f1_index = np.argmax(f1)

        opt_recall.append(recall[f1_index])
        opt_precision.append(precision[f1_index])

        t = thresh[f1_index - 1] if f1_index != 0 else thresh[0] - 1e-10
        opt_threshold.append(t)

    return np.array(opt_precision), np.array(opt_recall), np.array(opt_threshold)

In [ ]:
# Constants
score_fun = {"Precision": precision_score, "Recall": recall_score,
             "Specificity": partial(recall_score, pos_label=0), "F1 score": f1_score}

predictor_names = ["DNN", "cardio.", "emerg.", "stud."]
diagnosis = ["1dAVb", "RBBB", "LBBB", "SB", "AF", "ST"]
nclasses = len(diagnosis)

In [ ]:
# Read datasets

# Get cardiologists, residents and students performance
y_gold = pd.read_csv(TEST_ANNOTATIONS_PATH / "gold_standard.csv").values
y_student = pd.read_csv(TEST_ANNOTATIONS_PATH / "medical_students.csv").values
y_emerg = pd.read_csv(TEST_ANNOTATIONS_PATH / "emergency_residents.csv").values
y_cardio = pd.read_csv(TEST_ANNOTATIONS_PATH / "cardiology_residents.csv").values

y_dnn = np.load(PREDICTIONS_PATH / f"prediction_{model_id}.npy")

In [ ]:
# Get average model

# Get micro average precision
micro_avg_precision = average_precision_score(y_gold[:, :6], y_dnn[:, :6], average="micro")
print(f"Micro average precision: {micro_avg_precision}")

# Get threshold that yield the best precision recall using
# "get_optimal_precision_recall" on validation set
threshold = np.array([0.124, 0.07, 0.05, 0.278, 0.390, 0.174])
mask = y_dnn > threshold

# Get neural network prediction
y_neuralnet = np.zeros_like(y_dnn)
y_neuralnet[mask] = 1

In [ ]:
# Generate table with scores for the average model (Table 2)

scores_list = []
for y_pred in [y_neuralnet, y_cardio, y_emerg, y_student]:
    scores = get_scores(y_gold, y_pred, score_fun)
    scores_df = pd.DataFrame(scores, index=diagnosis, columns=score_fun.keys())
    scores_list.append(scores_df)

# Concatenate dataframes
scores_all_df = pd.concat(scores_list, axis=1, keys=predictor_names)
scores_all_df = scores_all_df.swaplevel(0, 1, axis=1)
scores_all_df = scores_all_df.reindex(level=0, columns=score_fun.keys())

# Save results
scores_all_df.to_csv(FIGURES_PATH / f"scores_{model_id}.csv", float_format="%.3f")


In [ ]:
# Compute scores and bootstrapped version of these scores

bootstrap_nsamples = 1000
percentiles = [2.5, 97.5]
scores_resampled_list = []
scores_percentiles_list = []
for y_pred in [y_neuralnet, y_cardio, y_emerg, y_student]:
    # Compute bootstrapped samples
    np.random.seed(123)  # NEVER change this
    n, _ = np.shape(y_gold)
    samples = np.random.randint(n, size=n * bootstrap_nsamples)

    # Get samples
    y_true_resampled = np.reshape(y_gold[samples, :], (bootstrap_nsamples, n, nclasses))
    y_doctors_resampled = np.reshape(y_pred[samples, :], (bootstrap_nsamples, n, nclasses))

    # Apply functions
    scores_resampled = np.array(
        [get_scores(y_true_resampled[i, :, :], y_doctors_resampled[i, :, :], score_fun)
         for i in range(bootstrap_nsamples)]
    )

    # Sort and append scores
    scores_resampled.sort(axis=0)
    scores_resampled_list.append(scores_resampled)

    # Get percentiles
    i = [int(p / 100.0 * bootstrap_nsamples) for p in percentiles]
    scores_percentiles = scores_resampled[i, :, :]
    scores_percentiles_df = pd.concat([pd.DataFrame(x, index=diagnosis, columns=score_fun.keys())
                                       for x in scores_percentiles], keys=["p1", "p2"], axis=1)
    scores_percentiles_df = scores_percentiles_df.swaplevel(0, 1, axis=1)
    scores_percentiles_df = scores_percentiles_df.reindex(level=0, columns=score_fun.keys())
    scores_percentiles_list.append(scores_percentiles_df)

# Concatenate dataframes
scores_percentiles_all_df = pd.concat(scores_percentiles_list, axis=1, keys=predictor_names)
scores_percentiles_all_df = scores_percentiles_all_df.reorder_levels([1, 0, 2], axis=1)
scores_percentiles_all_df = scores_percentiles_all_df.reindex(level=0, columns=score_fun.keys())

In [ ]:
# Print box plot (Supplementary Figure 1)

# Convert to xarray
scores_resampled_xr = xr.DataArray(
    np.array(scores_resampled_list),
    dims=["predictor", "n", "diagnosis", "score_fun"],
    coords={
        "predictor": predictor_names,
        "n": range(bootstrap_nsamples),
        "score_fun": list(score_fun.keys()),
        "diagnosis": diagnosis,
    }
)

for sf in score_fun:
    fig, ax = plt.subplots()
    f1_score_resampled_xr = scores_resampled_xr.sel(score_fun=sf)
    f1_score_resampled_df = f1_score_resampled_xr.to_dataframe(name=sf).reset_index(level=[0, 1, 2])

    # Plot seaborn
    sns.boxplot(x="diagnosis", y=sf, hue="predictor", data=f1_score_resampled_df, ax=ax)

    # Save results
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)

    plt.xlabel("")
    plt.ylabel("", fontsize=16)

    if sf == "F1 score":
        plt.legend(fontsize=17)
    else:
        ax.legend().remove()

    plt.tight_layout()
    plt.savefig(FIGURES_PATH / f"boxplot_bootstrap_{sf.lower()}_{model_id}.pdf")

(scores_resampled_xr.to_dataframe(name="score")
 .to_csv(FIGURES_PATH / f"boxplot_bootstrap_data_{model_id}.txt"))
